In [1]:
import torch
import torch.nn as nn
from brt.runtime import log
from brt.app.rand import RandScatter
from brt.router import GatherRouter


class MoE(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_scatter = RandScatter(path_num=2)
        self.expert1 = nn.Identity()
        self.expert2 = nn.Identity()
        self.gather_router = GatherRouter()
        self.iteration = 1
        self.ret = 1

    def forward(self, x):
        route_results = self.rand_scatter(x)
        x_0 = self.expert1(route_results[0])
        x_1 = self.expert2(route_results[1])
        x = self.gather_router([x_0, x_1])
        return x

class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.moe = MoE()

    def forward(self, x):
        x = self.moe(x)
        return x


moe_model = SimpleModel()

indata = torch.arange(0, 40, dtype=torch.float32).view(4, 10)
outdata = moe_model(indata)
print(outdata)


tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
        [10., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
        [20., 21., 22., 23., 24., 25., 26., 27., 28., 29.],
        [30., 31., 32., 33., 34., 35., 36., 37., 38., 39.]])


In [2]:
from brt.trace.graph import GraphTracer
from torch.fx.graph_module import GraphModule
from brt.runtime import BRT_CACHE_PATH
tracer = GraphTracer()
graph = tracer.trace(moe_model)
name = moe_model.__class__.__name__ if isinstance(moe_model, torch.nn.Module) else moe_model.__name__
graph_module= GraphModule(tracer.root, graph, name)

from torch.fx.passes.graph_drawer import FxGraphDrawer

graph_drawer = FxGraphDrawer(graph_module, "brt_model")
with open("a.svg", "wb") as f:
    f.write(graph_drawer.get_dot_graph().create_svg())




In [3]:
print(graph_module.code)
models = graph_module.named_modules()
# for node in graph.nodes:
#     print(node.target, node.args , node.users)

for node in graph.nodes:
    if node.target == "moe.gather_router":
        print(node.args)
        new_args = ([node.args[0][1]],)
        node.args = new_args
        print(node.args)

graph.eliminate_dead_code()
new_graph_module = GraphModule(tracer.root, graph, name)

print(new_graph_module.code)


from torch.fx.passes.graph_drawer import FxGraphDrawer

graph_drawer = FxGraphDrawer(new_graph_module, "new_brt_model")
with open("b.svg", "wb") as f:
    f.write(graph_drawer.get_dot_graph().create_svg())



torch.fx._symbolic_trace.wrap("brt_app_rand_rand_gate")

def forward(self, x):
    rand_gate = brt_app_rand_rand_gate(x, 2)
    moe_rand_scatter_scatter_router = self.moe.rand_scatter.scatter_router(x, rand_gate);  x = rand_gate = None
    getitem = moe_rand_scatter_scatter_router[0]
    moe_expert1 = self.moe.expert1(getitem);  getitem = None
    getitem_1 = moe_rand_scatter_scatter_router[1];  moe_rand_scatter_scatter_router = None
    moe_expert2 = self.moe.expert2(getitem_1);  getitem_1 = None
    moe_gather_router = self.moe.gather_router([moe_expert1, moe_expert2]);  moe_expert1 = moe_expert2 = None
    return moe_gather_router
    
([moe_expert1, moe_expert2],)
([moe_expert2],)

torch.fx._symbolic_trace.wrap("brt_app_rand_rand_gate")

def forward(self, x):
    rand_gate = brt_app_rand_rand_gate(x, 2)
    moe_rand_scatter_scatter_router = self.moe.rand_scatter.scatter_router(x, rand_gate);  x = rand_gate = None
    getitem_1 = moe_rand_scatter_scatter_router[1];  moe_rand_scatte

In [4]:
from brt.trace.graph import GraphTracer
from torch.fx.graph_module import GraphModule
from brt.runtime import BRT_CACHE_PATH
tracer = GraphTracer()
graph = tracer.trace(moe_model)
name = moe_model.__class__.__name__
graph_module= GraphModule(tracer.root, graph, name)
models = graph_module.named_modules()

for node in graph.nodes:
    if node.target == "moe.gather_router":
        new_args = ([node.args[0][1]],)
        node.args = new_args
        print(node.args)

graph.eliminate_dead_code()
new_graph_module = GraphModule(tracer.root, graph, name)
indata = torch.arange(0, 40, dtype=torch.float32).view(4, 10)
outdata = new_graph_module(indata)
print(outdata)

([moe_expert2],)
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
        [10., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])


In [12]:
import torch
import torch.nn as nn
from brt.runtime import log
from brt.app.rand import RandScatter
from brt.router import GatherRouter


class MoE(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_scatter = RandScatter(path_num=2, capturing=True)
        self.expert1 = nn.Identity()
        self.expert2 = nn.Identity()
        self.gather_router = GatherRouter(
            fabric_kwargs={"sparse": True}, capturing=True
        )
        self.iteration = 1
        self.ret = 1

    def forward(self, x):
        route_results = self.rand_scatter(x)
        x_0 = self.expert1(route_results[0])
        x_1 = self.expert2(route_results[1])
        x = self.gather_router([x_0, x_1])
        return x


class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.moe = MoE()

    def forward(self, x):
        x = self.moe(x)
        return x


moe_model = SimpleModel()

indata = torch.arange(0, 40, dtype=torch.float32).view(4, 10)
outdata = moe_model(indata)
print(outdata)
print(moe_model.moe.rand_scatter.scatter_router.load_history)
print(moe_model.moe.gather_router.load_history)


tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
        [10., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
        [20., 21., 22., 23., 24., 25., 26., 27., 28., 29.],
        [30., 31., 32., 33., 34., 35., 36., 37., 38., 39.]])
tensor([0., 4.], dtype=torch.float64)
tensor([0., 4.], dtype=torch.float64)


In [15]:
from brt.passes import get_pass

eliminate_pass_cls = get_pass("dead_path_eliminate")
eliminate_pass = eliminate_pass_cls(moe_model)
eliminate_pass.run_on_graph()
new_moe_model = eliminate_pass.finalize()
print(new_moe_model.code)
indata = torch.arange(0, 40, dtype=torch.float32).view(4, 10)
outdata = new_moe_model(indata)
print(outdata)



torch.fx._symbolic_trace.wrap("brt_app_rand_rand_gate")

def forward(self, x):
    rand_gate = brt_app_rand_rand_gate(x, 2)
    moe_rand_scatter_scatter_router = self.moe.rand_scatter.scatter_router(x, rand_gate);  x = rand_gate = None
    getitem = moe_rand_scatter_scatter_router[0];  moe_rand_scatter_scatter_router = None
    moe_expert1 = self.moe.expert1(getitem);  getitem = None
    moe_gather_router = self.moe.gather_router([moe_expert1]);  moe_expert1 = None
    return moe_gather_router
    
ProtoTensor([[10., 11., 12., 13., 14., 15., 16., 17., 18., 19.]])
tag_stack: [tensor([[1]])]
load stack: [4])


In [3]:
import torch
import torch.nn as nn
import torch.fx as fx
from brt.runtime.preload import EventEmitter, EventCollector, PreLoader
from brt.trace.graph import symbolic_trace

class SimpleNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.event_emitter = EventEmitter(2)
        self.event_collector = EventCollector(2)
        self.indentity = nn.Identity()

    def forward(self, x):
        x, e1, e2 = self.event_emitter(x)
        x = self.indentity(x)
        y = self.event_collector(x, e1, e2)
        return y

simple_net = SimpleNet()
x = torch.randn(2, 3)
print(x)
y = simple_net(x)
print(y)

g = symbolic_trace(SimpleNet())
print(g.graph)
g.graph.eliminate_dead_code()
g.recompile()
print(g.graph)

from torch.fx.passes.graph_drawer import FxGraphDrawer

graph_drawer = FxGraphDrawer(g, "preload_model")
with open("preload_model.svg", "wb") as f:
    f.write(graph_drawer.get_dot_graph().create_svg())



tensor([[ 1.0143, -0.5633,  0.4272],
        [-0.5826, -1.2026,  0.7366]])
tensor([[ 1.0143, -0.5633,  0.4272],
        [-0.5826, -1.2026,  0.7366]])
graph():
    %x : [#users=1] = placeholder[target=x]
    %event_emitter : [#users=3] = call_module[target=event_emitter](args = (%x,), kwargs = {})
    %getitem : [#users=1] = call_function[target=operator.getitem](args = (%event_emitter, 0), kwargs = {})
    %getitem_1 : [#users=1] = call_function[target=operator.getitem](args = (%event_emitter, 1), kwargs = {})
    %getitem_2 : [#users=1] = call_function[target=operator.getitem](args = (%event_emitter, 2), kwargs = {})
    %indentity : [#users=1] = call_module[target=indentity](args = (%getitem,), kwargs = {})
    %event_collector : [#users=1] = call_module[target=event_collector](args = (%indentity, %getitem_1, %getitem_2), kwargs = {})
    return event_collector
graph():
    %x : [#users=1] = placeholder[target=x]
    %event_emitter : [#users=3] = call_module[target=event_emitter](arg